<a href="https://colab.research.google.com/github/Frederik-Roeckle/xwines_recom/blob/master/Recommender_Models/Graphs/LightGCN_XWines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

!pip install pyg-lib==0.4.0+pt26cu124 \
            torch-scatter==2.1.2+pt26cu124 \
            torch-sparse==0.6.18+pt26cu124 \
            torch-cluster==1.6.3+pt26cu124 \
            torch-spline-conv==1.2.2+pt26cu124 \
            -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

!pip install torch-geometric

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 134.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidi

In [ ]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.utils import structured_negative_sampling
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm
import pathlib as pl
from sentence_transformers import SentenceTransformer
import math
import copy
import datetime
from torch_geometric.nn import LightGCN
from torch_geometric.utils import degree
import os.path as osp
from torch_geometric.utils import to_undirected, train_test_split_edges
from torch_geometric.data import Data

In [ ]:
# Get PyTorch version
pytorch_version = torch.__version__
print(f"PyTorch version: {pytorch_version}")

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

# Get CUDA version (if available)
if cuda_available:
    cuda_version = torch.version.cuda
    print(f"CUDA version: {cuda_version}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4


In [ ]:
csv_wines = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/XWines_Full_100K_wines.csv")
csv_training = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/trainset.csv")

reviews = pd.read_csv(csv_training)

# for shorter training time prototyping reduce trainset size
#split = 0.3
#reviews = reviews.loc[reviews.index < len(reviews) * split]

all_wines = pd.read_csv(csv_wines)

wines = all_wines.loc[all_wines["WineID"].isin(reviews["WineID"])]

In [ ]:
user_encoder = LabelEncoder()
wine_encoder = LabelEncoder()

reviews['UserID_idx'] = user_encoder.fit_transform(reviews['UserID'])
reviews['WineID_idx'] = wine_encoder.fit_transform(reviews['WineID'])

num_users = len(user_encoder.classes_)
num_wines = len(wine_encoder.classes_)

data = HeteroData()
data['user'].num_nodes = reviews['UserID'].nunique()
data['wine'].num_nodes = reviews['WineID'].nunique()
data['user', 'rates', 'wine'].edge_index = torch.tensor([
    reviews['UserID_idx'].values,
    reviews['WineID_idx'].values
], dtype=torch.long)

# Add ratings as edge weights
data['user', 'rates', 'wine'].edge_attr = torch.tensor(reviews['Rating'].values, dtype=torch.float).unsqueeze(1)

# Convert HeteroData to homogeneous bipartite graph (user-item -> single edge_index)
edge_index = data['user', 'rates', 'wine'].edge_index
num_users = data['user'].num_nodes
num_wines = data['wine'].num_nodes
num_nodes = num_users + num_wines

# Map edge_index to unified node space
user_offset = 0
wine_offset = num_users
edge_index_hom = edge_index.clone()
edge_index_hom[0] += user_offset
edge_index_hom[1] += wine_offset
print(edge_index_hom)

# Make it undirected
edge_index_hom = to_undirected(edge_index_hom)
# print(edge_index_hom)

# Prepare node features (can be zeros; LightGCN uses embeddings)
x = torch.zeros((num_nodes, 1))  # Dummy features

# Create homogeneous Data object
homo_data = Data(edge_index=edge_index_hom, x=x)
print(homo_data["edge_index"])


<ipython-input-6-f4c1bd0fb3ef>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  data['user', 'rates', 'wine'].edge_index = torch.tensor([


tensor([[ 392814,  392814,  383338,  ..., 1056007, 1056023, 1056023],
        [1091528, 1129108, 1119940,  ..., 1135679, 1128827, 1129294]])
tensor([[      0,       0,       0,  ..., 1154859, 1154859, 1154859],
        [1056600, 1057519, 1057557,  ...,  479663,  813430,  898817]])


In [ ]:
from torch.optim import Adam


# Make sure your data is on CPU first (LinkNeighborLoader handles moving to device)
# `homo_data` should be a torch_geometric.data.Data object with .edge_index
# edge_label_index = edges you want to predict (typically all edges in training)

# Train/val split (simple 90/10)
num_edges = homo_data.edge_index.size(1)
perm = torch.randperm(num_edges)
train_edge_index = homo_data.edge_index[:, perm[:int(0.9 * num_edges)]]
val_edge_index = homo_data.edge_index[:, perm[int(0.9 * num_edges):]]

# Define the LightGCN model
embedding_dim = 64
num_layers = 3
num_nodes = homo_data.num_nodes

model = LightGCN(num_nodes=num_nodes, embedding_dim=embedding_dim, num_layers=num_layers).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

# Define DataLoaders
train_loader = LinkNeighborLoader(
    data=homo_data,
    edge_label_index=train_edge_index,
    edge_label=torch.ones(train_edge_index.size(1)),  # dummy labels
    num_neighbors=[20, 10],  # sample 2 hops (you can tune this)
    batch_size=8192,
    shuffle=True,
    neg_sampling='binary',
    neg_sampling_ratio=1.0,  # 1:1 pos-neg for BPR
)

val_loader = LinkNeighborLoader(
    data=homo_data,
    edge_label_index=val_edge_index,
    edge_label=torch.ones(val_edge_index.size(1)),
    num_neighbors=[20, 10],
    batch_size=8192,
    shuffle=False,
    neg_sampling_ratio=1.0,
)


In [ ]:
from torch.nn.functional import sigmoid

def bpr_loss(pos_scores, neg_scores):
    return -torch.log(sigmoid(pos_scores - neg_scores)).mean()

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()

        # Get embeddings from model
        z = model(batch.edge_index)

        # Sample positive pairs from batch
        user_nodes = batch.edge_label_index[0]
        pos_item_nodes = batch.edge_label_index[1]

        # Sample random negative items for each user
        neg_item_nodes = torch.randint(low=num_users, high=num_nodes -1, size=pos_item_nodes.size(), device=device)

        # Get embeddings
        user_emb = z[user_nodes]
        pos_item_emb = z[pos_item_nodes]
        neg_item_emb = z[neg_item_nodes]

        # Dot product scores
        pos_scores = (user_emb * pos_item_emb).sum(dim=-1)
        neg_scores = (user_emb * neg_item_emb).sum(dim=-1)

        # BPR loss
        loss = bpr_loss(pos_scores, neg_scores)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


100%|██████████| 3652/3652 [29:13<00:00,  2.08it/s]


Epoch 1, Loss: 171.8234


100%|██████████| 3652/3652 [29:06<00:00,  2.09it/s]


Epoch 2, Loss: 0.0000


100%|██████████| 3652/3652 [29:04<00:00,  2.09it/s]

Epoch 3, Loss: 0.0000


In [ ]:
model.eval()
with torch.no_grad():
    total_val_loss = 0
    for batch in val_loader:
        batch = batch.to(device)
        z = model(batch.edge_index)

        src = batch.edge_label_index[0]
        pos_dst = batch.edge_label_index[1]
        neg_dst = batch.edge_label_index_neg[1]

        pos_score = (z[src] * z[pos_dst]).sum(dim=-1)
        neg_score = (z[src] * z[neg_dst]).sum(dim=-1)

        loss = bpr_loss(pos_score, neg_score)
        total_val_loss += loss.item()

    print(f"Validation Loss: {total_val_loss / len(val_loader):.4f}")


In [ ]:
from torch_geometric.utils import negative_sampling


# Assume you already have these from preprocessing:
# homo_data.edge_index, num_nodes, num_users, num_wines

# Step 1: Manual split of the edge index
edge_index = homo_data.edge_index
num_edges = edge_index.size(1)
perm = torch.randperm(num_edges)

train_ratio = 0.8
val_ratio = 0.1

num_train = int(train_ratio * num_edges)
num_val = int(val_ratio * num_edges)

train_edge_index = edge_index[:, perm[:num_train]]
val_edge_index = edge_index[:, perm[num_train:num_train + num_val]]
test_edge_index = edge_index[:, perm[num_train + num_val:]]

# Prepare the training graph
train_data = homo_data.clone()
train_data.edge_index = train_edge_index
train_data = train_data.to(device)

# Step 2: Prepare DataLoaders for edges
batch_size = 8192
train_loader = torch.utils.data.DataLoader(
    range(train_edge_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

val_loader = torch.utils.data.DataLoader(
    range(val_edge_index.size(1)),
    shuffle=False,
    batch_size=batch_size,
)

# Step 3: Initialize model
embedding_dim = 64
num_layers = 3
model = LightGCN(num_nodes=num_nodes, embedding_dim=embedding_dim, num_layers=num_layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Step 4: Train
epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_indices in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch_indices = torch.tensor(batch_indices, dtype=torch.long, device=device)
        pos_edges = train_data.edge_index[:, batch_indices]

        # Sample negatives
        neg_edges = negative_sampling(
            edge_index=train_edge_index,
            num_nodes=num_nodes,
            num_neg_samples=pos_edges.size(1),
            method='sparse'
        ).to(device)

        # Combine for scoring
        edge_label_index = torch.cat([pos_edges, neg_edges], dim=1)
        scores = model(train_data.edge_index, edge_label_index)
        pos_scores, neg_scores = scores[:pos_edges.size(1)], scores[pos_edges.size(1):]

        # BPR Loss
        loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")


Epoch 1:   0%|          | 0/3247 [00:00<?, ?it/s]<ipython-input-9-677217fa2a51>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_indices = torch.tensor(batch_indices, dtype=torch.long, device=device)
Epoch 1:   6%|▌         | 182/3247 [07:38<2:08:38,  2.52s/it]


KeyboardInterrupt: 

In [ ]:
# Assuming your HeteroData object has been preprocessed correctly and moved to device
data = data.to(device)  # `data` is your HeteroData object
homo_data = homo_data.to(device)

# Parameters
embedding_dim = 64
num_layers = 3

# Initialize LightGCN
model = LightGCN(num_nodes=num_nodes, embedding_dim=embedding_dim, num_layers=3).to(device)

# BPR loss setup (positive and negative sampling)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# Use all message passing edges as training labels:
batch_size = 8192
mask = homo_data.edge_index[0] < homo_data.edge_index[1]
train_edge_label_index = homo_data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model.train()
for epoch in tqdm(train_loader):
    optimizer.zero_grad()

    # Sample positive edges (user, item) pairs
    edge = data['user', 'rates', 'wine'].edge_index
    num_samples = edge.size(1)

    pos_user = edge[0].to(device)
    pos_item = edge[1].to(device) + num_users  # offset wine node indices

    # Sample random negative items (same users, different items)
    neg_item = torch.randint(0, num_wines, (num_samples,), device=device) + num_users

    # Create edge index for prediction
    pos_edge_label_index = torch.stack([pos_user, pos_item])
    neg_edge_label_index = torch.stack([pos_user, neg_item])
    edge_label_index = torch.cat([pos_edge_label_index, neg_edge_label_index], dim=1)

    # Forward pass: compute scores for all (pos + neg) pairs
    scores = model(homo_data.edge_index, edge_label_index)
    pos_scores, neg_scores = scores.chunk(2, dim=0)

    # BPR loss
    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()

    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

  0%|          | 1/613 [00:00<09:23,  1.09it/s]

Epoch tensor([4445568, 2832914,  783291,  ..., 3934314, 1321286, 4999498]), Loss: 0.6931


  0%|          | 2/613 [00:01<07:00,  1.45it/s]

Epoch tensor([4452847, 2618387, 3726417,  ..., 2259048, 4384926, 3111528]), Loss: 0.6931


  0%|          | 3/613 [00:01<06:13,  1.63it/s]

...

Epoch tensor([2137945, 1111222, 4159633,  ...,  244234, 1586778, 4559461]), Loss: 0.2819


100%|█████████▉| 612/613 [05:22<00:00,  1.90it/s]

Epoch tensor([1132822, 1674050, 3304709,  ..., 4892287, 1139383, 2607321]), Loss: 0.2815


100%|██████████| 613/613 [05:23<00:00,  1.90it/s]

Epoch tensor([3074307, 1142492, 2419711, 2312116, 4383903, 4266790,  431826, 1607125,
        2955783, 4959489, 3186796, 3406742,  245089, 1474941, 2278709, 4072201,
        3301846, 4595479, 2186910, 3418002,  823180,  340378,  938830, 2498342,
        4185125, 1977489, 4975642, 1718561, 3262950,  794380,  691232, 2121366,
        3597301, 1257276,  332305,  311156, 3626341, 3215505, 2063877, 4454168,
        4722780, 2864830,  173562, 3521775, 1606213,  827458, 1214493, 2049257,
        2937154, 3557644, 3025048, 4142864, 4164623, 3253878, 1748603, 4976954,
        3474563, 4427595, 4041687, 1287545, 3970272, 1611218, 2132484, 4100508,
        4402379, 2564834, 1470370, 4481299, 4233334, 3124675, 1598061, 4730460,
        1503108, 4854845, 4561124, 3772321, 2676159, 3172710, 4103075, 3315469,
        3036254, 2093814, 2328580, 1126288, 3829120,  615793,  751751, 1178516,
        1915294,  856177, 1066850, 1079841, 3753972, 2043910,  147520, 2777359,
        4997016, 2588776, 2596472,

In [ ]:
# Assuming your HeteroData object has been preprocessed correctly and moved to device
data = data.to(device)  # `data` is your HeteroData object
homo_data = homo_data.to(device)

# Parameters
embedding_dim = 64
num_layers = 3

# Initialize LightGCN
model = LightGCN(num_nodes=num_nodes, embedding_dim=embedding_dim, num_layers=3).to(device)

# BPR loss setup (positive and negative sampling)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# Use all message passing edges as training labels:
batch_size = 8192
mask = homo_data.edge_index[0] < homo_data.edge_index[1]
train_edge_label_index = homo_data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model.train()
for batch in tqdm(train_loader):
    optimizer.zero_grad()

    # Sample batch of edges (positive)
    pos_edges = train_edge_label_index[:, batch]
    pos_user = pos_edges[0]
    pos_item = pos_edges[1]

    # Negative sampling (avoid sampling positive items)
    neg_item = torch.randint(0, num_wines, (pos_user.size(0),), device=device)

    # Offset wine indices if needed
    #pos_item = pos_item + num_users
    #neg_item = neg_item + num_users

    # Build edge index for LightGCN forward
    edge_label_index = torch.cat([
        torch.stack([pos_user, pos_item]),
        torch.stack([pos_user, neg_item])
    ], dim=1)

    print("edge_label_index max:", edge_label_index.max().item())
    print("embedding size:", model.embedding.weight.size(0))


    # Compute scores
    scores = model(homo_data.edge_index, edge_label_index)
    pos_scores, neg_scores = scores.chunk(2, dim=0)

    # BPR loss
    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()

    loss.backward()
    optimizer.step()

    print(f"Batch Loss: {loss.item():.4f}")


  0%|          | 0/613 [00:00<?, ?it/s]

edge_label_index max: 575701
embedding size: 575890


  0%|          | 1/613 [00:00<07:10,  1.42it/s]

Batch Loss: 0.6931
edge_label_index max: 575872
embedding size: 575890


  0%|          | 2/613 [00:01<05:07,  1.99it/s]

...

Batch Loss: 0.0722
edge_label_index max: 575498
embedding size: 575890


100%|█████████▉| 612/613 [03:41<00:00,  2.76it/s]

Batch Loss: 0.0678
edge_label_index max: 574381
embedding size: 575890


100%|██████████| 613/613 [03:42<00:00,  2.75it/s]

Batch Loss: 0.0721
